# **(Data-Understanding and Further Cleaning)**

## Objectives

* Discover how various house attributes correlate with the final sale price. (Business Requirement 1)

* Analyse variables containing missing values and fill them in appropriately

## Inputs

* outputs/datasets/collection/inherited.csv

* outputs/datasets/collection/original.csv

## Outputs

* Relevant plots so the client can quickly visualize the results of the correlation analysis. (How certain features may impact the sale price)

* outputs/datasets/collection/original_no_nan.csv

* outputs/datasets/collection/further_analysis.csv


---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

## Load Data

In [ ]:
import pandas as pd

df_original = pd.read_csv("outputs/datasets/collection/original.csv")

df_original.head(5)

---

# Pandas Profiling Report

In [ ]:
from pandas_profiling import ProfileReport

report_on_house_records = ProfileReport(df=df_original, title='An overview of the original dataset', minimal=True)
report_on_house_records.to_notebook_iframe()


---

- To OneHotEncode categorical variables, missing values will need to be removed.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_original.head(8)

In [ ]:
df_original.info()


###     Assess the variables with missing values to determine if/how to fill them or drop them.

- After looking at the BsmtFinType1 one column again I replaced the nan values with 'none'. I felt in this dataset they were saying the same thing, i.e. there were no finishes in the basement.

    - I plotted a bar plot to see what the distribution was like before and after the change.

In [ ]:
columns_with_nan = df_original.columns[df_original.isna().sum() > 0].to_list()
columns_with_nan

df_original[columns_with_nan].isna().sum()

In [ ]:
print(df_original['BsmtFinType1'].value_counts())
print(df_original['BsmtFinType1'].unique())

In [ ]:
df_original.value_counts('BsmtFinType1').plot(kind='bar', figsize=(10,4))
plt.show()

In [ ]:
df_original['BsmtFinType1'] = df_original['BsmtFinType1'].fillna(value='None')

In [ ]:
print(df_original['BsmtFinType1'].value_counts())
print(df_original['BsmtFinType1'].unique())

In [ ]:
df_original.value_counts('BsmtFinType1').plot(kind='bar', figsize=(10,4))
plt.show()

- I approached GarageFinish in the same way and grouped the nan values into the 'None' category

In [ ]:
print(df_original['GarageFinish'].value_counts())
print(df_original['GarageFinish'].unique())

In [ ]:
df_original.value_counts('GarageFinish').plot(kind='bar', figsize=(10,4))

In [ ]:
df_original['GarageFinish'] = df_original['GarageFinish'].fillna(value='None')
print(df_original['GarageFinish'].value_counts())
print(df_original['GarageFinish'].unique())

In [ ]:
df_original.value_counts('GarageFinish').plot(kind='bar', figsize=(10,4))

- From looking at the histogram in the Pandas profiling report above for GarageYrBlt, I could see that the distribution of values was not normal. 

- The mean and median are close in terms of when the properties were built so I will fill nan values in this catgory with the median value for this category.

In [ ]:
print(df_original['GarageYrBlt'].mean())
print(df_original['GarageYrBlt'].median())

In [ ]:
df_original['GarageYrBlt'].plot(kind='hist', y='GarageYrBlt', bins=10)

In [ ]:
columns_with_nan = df_original.columns[df_original.isna().sum() > 0].to_list()
columns_with_nan

In [ ]:
df_original['GarageYrBlt'] = df_original['GarageYrBlt'].fillna(df_original['GarageYrBlt'].median())

In [ ]:
columns_with_nan = df_original.columns[df_original.isna().sum() > 0].to_list()
columns_with_nan

In [ ]:
df_original['GarageYrBlt'].plot(kind='hist', y='GarageYrBlt', bins=10)

- Lot Frontage: I filled the nan values with the median again here. Histograms plotted to show the before and after effects.

In [ ]:
df_original['LotFrontage'].plot(kind='hist', y='LotFrontage', bins=10)

In [ ]:
print(df_original['LotFrontage'].mean())
print(df_original['LotFrontage'].median())

In [ ]:
df_original['LotFrontage'] = df_original['LotFrontage'].fillna(df_original['LotFrontage'].median())

In [ ]:
df_original['LotFrontage'].plot(kind='hist', y='LotFrontage', bins=10)
print(df_original['LotFrontage'].mean())
print(df_original['LotFrontage'].median())

#### Categorical variables can now be OneHotEncoded

In [ ]:
from feature_engine.encoding import OneHotEncoder

encoder = OneHotEncoder(variables=df_original.columns[df_original.dtypes == 'object'].tolist(), drop_last=False)

encoded_df = encoder.fit_transform(df_original)

print(encoded_df.shape)

encoded_df.head(5)

In [ ]:
spearman_correlation = encoded_df.corr(method='spearman')['SalePrice'].sort_values(key=abs, ascending=False)[1:].head(10)
spearman_correlation 

In [ ]:
pearson_correlation = encoded_df.corr(method='pearson')['SalePrice'].sort_values(key=abs, ascending=False)[1:].head(10)
pearson_correlation 

In [ ]:
df_original['KitchenQual'].unique()

#### Observations from the spearmans and pearsons tests.

- OverallQual and GrLivArea look highly correlated with sale price which at this stage looks to strengthen the initial hypothesis

- KitchenQual which is average shows negative correlation with sale price

- KitchenQual which is Gd/Ex is showing positive correlation. This ranking of KitchenQual is important to consider further

- I will consider further, all features that returned values above 0.5 for either test


In [ ]:
columns_of_interest = ['OverallQual', 'GrLivArea', 'YearBuilt', 'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'YearRemodAdd', 'GarageYrBlt', 'KitchenQual']

columns_of_interest

### EDA on selected variables

In [ ]:
df_further_analyis = df_original.filter(columns_of_interest + ['SalePrice'])
df_further_analyis.head(10)
print(df_further_analyis.info())

- Changing GarageYrBuilt data type here to keep things uniform.

In [ ]:
df_further_analyis['GarageYrBlt'] = df_further_analyis['GarageYrBlt'].astype('int')
print(df_further_analyis.info())

### *Plot variables for visualisation*

In [ ]:
col = df_further_analyis['KitchenQual'].unique()
col

In [ ]:
fig, ((ax1,ax2),(ax3,ax4)) = plt.subplots(nrows=2, ncols=2, figsize=(15, 5))
sns.countplot(data=df_further_analyis, x='KitchenQual',ax=ax1)
sns.scatterplot(data=df_further_analyis, x='KitchenQual', y='SalePrice', hue='OverallQual',ax=ax2)
sns.scatterplot(data=df_further_analyis, x='KitchenQual', y='SalePrice', hue='GrLivArea',ax=ax3)
sns.scatterplot(data=df_further_analyis, x='KitchenQual', y='SalePrice', hue='GarageArea',ax=ax4)
plt.tight_layout()
plt.show()

- The above plots show really good sale prices when houses contain 'Gd' 'Ex' grade of kitchens, coupled with high overall quality, good size living area, and have good garage area.

In [ ]:
fig = plt.figure(figsize=(20, 10))
sns.boxplot(data=df_further_analyis, x='OverallQual', y='SalePrice')


- This box plot shows how strong the correlation is between overall quality ratings between 8 and 10 and the eventual sale price. There are very few outliers in the negative direction in this group.

- The combined IQR for ratings 9 + 10 give an indication of a sale price between 320,000 and 480,000

In [ ]:
fig = plt.figure(figsize=(20, 10))
sns.violinplot(data=df_further_analyis, x=df_further_analyis['OverallQual'], y=df_further_analyis['SalePrice'], hue='KitchenQual')

- This vertical viloin plot shows how few houses with a kitchen quality rating of TA(Typical,Average) managed a decent sale price. Category 8 shows this particularly well.

- The overall quality was high but the sale price has a negative correlation with kitchen quality.

- There is a considerable jump in sale price when kitchen quality is excellent to compliment the overall house condition.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(20, 5))
axes[0].bar(data=df_further_analyis, x="OverallQual", height='SalePrice')
axes[1].bar(data=df_further_analyis, x="OverallQual", height='GrLivArea')
axes[2].bar(data=df_further_analyis, x="OverallQual", height='GarageArea')
axes[3].bar(data=df_further_analyis, x="OverallQual", height='TotalBsmtSF')

In [ ]:
fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize=(20,7))
sns.scatterplot(data=df_further_analyis, x='YearRemodAdd', y='SalePrice', hue='OverallQual', ax=ax1)
sns.scatterplot(data=df_further_analyis, x='YearBuilt', y='SalePrice', hue='OverallQual', ax=ax2)

- Modern houses showing correlation with sale price. Modern houses seem to have greater space and better finishesand thus a greater sale price.

In [ ]:
fig =  plt.figure(figsize=(15,5))
sns.scatterplot(data=df_further_analyis, x='YearBuilt', y='SalePrice', hue='1stFlrSF')

- The above plot shows a greater number of larger houses after about 1970. This correlates to eventual sale price.

### Current conclusions from this analysis:

- The overall quality rating seems to be a key influence on sale price.

- This rating seems to be influenced heavily by the year it was built and/or the year any remodelling took place.

- The overall quality rating is influenced/complimented by a kitchen quality rating of at least good, but moreso excellent.

- To ensure a good sale price:

    - An overall quality rating between 8-10
    - An excellent grade of kitchen
    - A garage area that is large and finished well
    - Generally, more modern and spacious properties seem to fetch higher values.

---

# Push files to Repo

* If you do not need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
df_original.to_csv(f"outputs/datasets/collection/original_no_nan.csv", index=False)
df_further_analyis.to_csv(f"outputs/datasets/collection/further_analyis.csv")
